## 1-1. Module import

In [1]:
from os import listdir
import os 
import pandas as pd
import numpy as np
from itertools import islice

In [2]:
def window(seq, n):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [3]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

## 1-2. IS 계산

In [4]:
def N_cal(fil):
    fil_tmp = fil.copy()
    
    fil_tmp["Time"] = pd.to_datetime(fil_tmp["Time"], errors='coerce')
    fil_tmp["Time"] = fil_tmp["Time"].dt.strftime('%Y-%m-%d %H:%M:%S')

    z = []
    for i in fil_tmp.iloc[:,0]:
        #print(i[0:10])
        z.append(i[0:10])
        #print(z)
    fil_tmp.iloc[:,0] = z
    #print(fil_tmp)

    fil_grb = fil_tmp.groupby('Time').agg('count')

    fil_2 = fil_grb.transpose()
    lst_fil_2 = list(fil_2)
    len_cd = len(lst_fil_2)
       
    n = 24 * len_cd
    
    return n

In [5]:
#전체 평균 구하는 함수
def ent_mean(fil):
    fil_tmp = fil.copy()
    fil_tmp_des = fil_tmp.describe()
    #print(fil_tmp_des)
    mean_fil = fil_tmp_des.loc['mean']
    lst_mean_fil = list(mean_fil)
    int_mean_fil = sum(lst_mean_fil)
    
    return int_mean_fil

In [6]:
def cal_ent_mean(df):
    ent_mean = df.Activity.mean()
    
    return ent_mean

In [7]:
def sum_square_up(fil):
    fil_tmp = fil.copy()

    fil_tmp['Time'] = pd.to_datetime(fil_tmp['Time'], errors='coerce')
    fil_tmp['Time'] = fil_tmp['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')

    z = []
    for i in fil_tmp.iloc[:,0]:
        z.append(i[10:13])
    fil_tmp.iloc[:,0] = z

    
    fil_tmp['Activity'] = fil_tmp['Activity'].apply(pd.to_numeric)
    time_mean = fil_tmp.groupby(['Time']).agg('mean')
    time_mean = time_mean['Activity'] - fil.Activity.mean()
    time_mean = time_mean.to_frame()
    time_mean = time_mean.reset_index()
    act_tmp = time_mean.drop(['Time'],1)
    act_tmp['Activity'] = act_tmp['Activity'] ** 2
    aaa = act_tmp.values.sum()
    
    return aaa   

In [8]:
def sum_cal_dw(fil):
    fil_tmp = fil.copy()
    fil_tmp_mean = fil_tmp.groupby(['datehour']).agg('mean')
    fil_tmp_mean = fil_tmp_mean['Activity'] - fil.Activity.mean()

    fil_tmp_mean = fil_tmp_mean.to_frame()
    fil_tmp_mean = fil_tmp_mean.reset_index()

    actm_tmp = fil_tmp_mean.drop(['datehour'],1)
    actm_tmp['Activity'] = actm_tmp['Activity'] ** 2
    aaa = actm_tmp.values.sum()

    aaa_tmp = aaa * 24
    
    return aaa_tmp

In [9]:
def final_IS_cal(fil):
    
    
    N = N_cal(fil)
    up_b = sum_square_up(fil)
    dw = sum_cal_dw(fil)
    
    IS = (N * up_b)  / dw
    
    return IS

## 1-3. IV 지표 계산!

In [10]:
def N_cal(fil):
    fil_tmp = fil.copy()
    
    fil_tmp["Time"] = pd.to_datetime(fil_tmp["Time"], errors='coerce')
    fil_tmp["Time"] = fil_tmp["Time"].dt.strftime('%Y-%m-%d %H:%M:%S')
    

    z = []
    for i in fil_tmp.iloc[:,0]:
        #print(i[0:10])
        z.append(i[0:10])
    fil_tmp.iloc[:,0] = z

    fil_grb = fil_tmp.groupby('Time').agg('count')
    #fil_2 = fil_grb.drop(['MET'],1)

    fil_2 = fil_grb.transpose()
    lst_fil_2 = list(fil_2)
    len_cd = len(lst_fil_2)
       
    n = 24 * len_cd
    
    return n

In [11]:
def dividend(fil):
    fil_tmp = fil.copy()
    fil_tmp['Time'] = pd.to_datetime(fil_tmp['Time'], errors='coerce')
    fil_tmp['Time'] = fil_tmp['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')

    z = []
    for i in fil_tmp.iloc[:,0]:
        z.append(i[:13])
    fil_tmp.iloc[:,0] = z
    time_mean = fil_tmp.groupby(['Time']).agg('mean')
    time_mean = time_mean.reset_index()
    time_mean['new_Activity'] = time_mean['Activity'].map(lambda x : x)
    time_mean.new_Activity = time_mean.new_Activity.shift(+1)
    time_mean = time_mean.dropna()

    time_mean = time_mean['Activity'] - time_mean['new_Activity']
    cal_tmp = (time_mean ** 2)
    n_val = N_cal(df)
    aaa = cal_tmp.values.sum() * n_val
    
    return aaa 

In [12]:
def divisor(fil):
    fil_tmp = fil.copy()
    fil_tmp.head()       

    fil_tmp_mean = fil_tmp.groupby(['datehour']).agg('mean')
    fil_tmp_mean = fil.Activity.mean() - fil_tmp_mean['Activity']
    fil_tmp_mean = fil_tmp_mean.to_frame()
    fil_tmp_mean = fil_tmp_mean.reset_index()
    actm_tmp = fil_tmp_mean.drop(['datehour'],1)
    actm_tmp['Activity'] = actm_tmp['Activity'] ** 2
    aaa = actm_tmp.values.sum()
    aaa_tmp = aaa * (N_cal(fil) - 1)
   
    return aaa_tmp

In [13]:
def final_IV_cal(fil):
    up_b = dividend(fil)
    dw = divisor(fil)
    
    IV = up_b  / dw
    
    return IV

## 1-4. RA 지표 계산!

#1) L5, L5_starttime 계산!

In [14]:
#find min average values!
def find_min_average(data_list):
    tmp_data_dict={}
    for each in window(data_list, 300):
        tmp_five_list = list(each)
        #print([np.float(x[1]) for x in tmp_five_list])
        tmp_act_value = np.mean([np.float(x[0]) for x in tmp_five_list])
        tmp_dict = {tmp_act_value : tmp_five_list}
        tmp_data_dict.update(tmp_dict)
        
    min_average = min(tmp_data_dict)
    find_values = tmp_data_dict[min_average]
    
    return min_average, find_values


#Calculate L5!
def fnd_low(fil):
    fil_tmp = fil.copy()
    
    fil_tmp['Time'] = pd.to_datetime(fil_tmp['Time'], errors='coerce')
    fil_tmp['Time'] = fil_tmp['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    fil_tmp['datehour'] = fil_tmp['Time'].map(lambda x : x[11:19])
    
    time_mean_ = fil_tmp.groupby(['datehour']).agg('mean').reset_index(drop=True)
    time_mean_['time'] = fil_tmp['datehour']
    
    time_act_list = time_mean_.values.tolist()+time_mean_.values.tolist()
    time_act_list_ = time_act_list[:1440+299]
        
    min_average, find_values = find_min_average(time_act_list_)
    
    return round(min_average,2), find_values[0][1]

#2) M10, M10_starttime 계산!

In [15]:
#find max average values!
def find_max_average(data_list):
    tmp_data_dict={}
    for each in window(data_list, 600):
        tmp_five_list = list(each)
        #print([np.float(x[1]) for x in tmp_five_list])
        tmp_act_value = np.mean([np.float(x[0]) for x in tmp_five_list])
        tmp_dict = {tmp_act_value : tmp_five_list}
        tmp_data_dict.update(tmp_dict)
        
    max_average = max(tmp_data_dict)
    find_values = tmp_data_dict[max_average]
    
    return max_average, find_values

#Calculate L5!
def fnd_max(fil):
    fil_tmp = fil.copy()
    
    fil_tmp['Time'] = pd.to_datetime(fil_tmp['Time'], errors='coerce')
    fil_tmp['Time'] = fil_tmp['Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    fil_tmp['datehour'] = fil_tmp['Time'].map(lambda x : x[11:19])
    
    time_mean_ = fil_tmp.groupby(['datehour']).agg('mean').reset_index(drop=True)
    time_mean_['time'] = fil_tmp['datehour']
    
    time_act_list = time_mean_.values.tolist()+time_mean_.values.tolist()
    time_act_list_ = time_act_list[:1440+599]
   
    max_average, find_values = find_max_average(time_act_list_)
    
    return round(max_average,2), find_values[0][1]

In [16]:
def final_cal_RA(df):
    L5, L5_starttime = fnd_low(df)
    M10, M10_starttime = fnd_max(df)
    
    RA = round( (M10 - L5) / (M10 + L5) ,2)
    return RA, L5, M10, L5_starttime, M10_starttime

In [17]:
#Calculate indicator and make list!
def cal_indi(df):
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    df['Time'] = df['Time'].astype(str)
    df['datehour'] = df['Time'].map(lambda x : x[:13])
    
    IS_value = round(final_IS_cal(df), 2)
    IV_value = round(final_IV_cal(df), 2)
    RA, L5, M10, L5_starttime, M10_starttime = final_cal_RA(df)
    
    value = [IS_value, IV_value, RA, L5, L5_starttime, M10, M10_starttime]
    
    return value

In [18]:
##지표계산하는 메인함수!
print("Calculate start!")

os.chdir("/Users/junggu/Desktop/")

search_directory = 'data2/'

filelist = listdir(search_directory)
#print(filelist)
#filelist = filelist[:]
#print(filelist)



final = []
for i,n in enumerate(filelist):
    print("\nfile No:",i)
    print("file Name:",n)
    
    df = pd.read_csv(search_directory+"/"+n)#.drop('MET',1).drop('Calorie',1).drop('Wear',1)
    print(len(df))
    print(df.head())
    try:
        df.columns=['Time','Activity']
        df = df.round()

        value = cal_indi(df)
        value_with_pid = [str(n)]+value
        #print(value)
        final.append(value_with_pid)
    except ValueError:
        print(n)
        continue
    
final_df = pd.DataFrame(final)
final_df.columns = ['p_id','IS','IV','RA', 'L5', 'L5_starttime', 'M10', 'M10_starttime']
#final_df
os.chdir("/Users/junggu/Desktop/")
final_df.to_csv("result_file.csv", index=False) 

Calculate start!

file No: 0
file Name: P_ABD-AJ-0101_31.csv
1440
                  Time  Activity
0  2017-12-05 00:00:38       0.0
1  2017-12-05 00:01:38       0.0
2  2017-12-05 00:02:38       0.0
3  2017-12-05 00:03:38       0.0
4  2017-12-05 00:04:38       0.0

file No: 1
file Name: P_ABD-AJ-0083_32.csv
1440
                  Time  Activity
0  2017-09-23 00:00:29    9040.0
1  2017-09-23 00:01:29    8196.0
2  2017-09-23 00:02:29    8568.0
3  2017-09-23 00:03:29    4948.0
4  2017-09-23 00:04:29    5275.0

file No: 2
file Name: P_ABD-AJ-0001_25.csv
1440
                  Time  Activity
0  2016-12-17 00:00:40       0.0
1  2016-12-17 00:01:40       0.0
2  2016-12-17 00:02:40       0.0
3  2016-12-17 00:03:40       0.0
4  2016-12-17 00:04:40       0.0


/Users/junggu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/Users/junggu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """



file No: 3
file Name: P_ABD-AJ-0108_15.csv
1440
                  Time  Activity
0  2018-03-18 00:00:43       6.0
1  2018-03-18 00:01:43       0.0
2  2018-03-18 00:02:43    2256.0
3  2018-03-18 00:03:43    9301.0
4  2018-03-18 00:04:43       8.0

file No: 4
file Name: P_ABD-AJ-0053_11.csv
1440
                  Time  Activity
0  2017-07-11 00:00:58       0.0
1  2017-07-11 00:01:58       0.0
2  2017-07-11 00:02:58       0.0
3  2017-07-11 00:03:58       0.0
4  2017-07-11 00:04:58       0.0

file No: 5
file Name: P_ABD-AJ-0029_3.csv
1440
                  Time  Activity
0  2017-04-25 00:00:39       0.0
1  2017-04-25 00:01:39       0.0
2  2017-04-25 00:02:39       0.0
3  2017-04-25 00:03:39       0.0
4  2017-04-25 00:04:39       0.0

file No: 6
file Name: P_ABD-AJ-0083_26.csv
1440
                  Time  Activity
0  2017-10-12 00:00:54    1537.0
1  2017-10-12 00:01:54    9395.0
2  2017-10-12 00:02:54   15013.0
3  2017-10-12 00:03:54    2700.0
4  2017-10-12 00:04:54       5.0

file No: 7
f


file No: 37
file Name: P_ABD-AJ-0094_16.csv
1440
                  Time  Activity
0  2017-12-31 00:00:47     379.0
1  2017-12-31 00:01:47     120.0
2  2017-12-31 00:02:47      16.0
3  2017-12-31 00:03:47       0.0
4  2017-12-31 00:04:47       0.0

file No: 38
file Name: P_ABD-AJ-0116_15.csv
1440
                  Time  Activity
0  2018-05-15 00:00:14       0.0
1  2018-05-15 00:01:14       0.0
2  2018-05-15 00:02:14       0.0
3  2018-05-15 00:03:14       0.0
4  2018-05-15 00:04:14     176.0

file No: 39
file Name: P_ABD-AJ-0043_28.csv
1440
                  Time  Activity
0  2017-06-14 00:00:28       0.0
1  2017-06-14 00:01:28       0.0
2  2017-06-14 00:02:28       0.0
3  2017-06-14 00:03:28       0.0
4  2017-06-14 00:04:28       0.0

file No: 40
file Name: P_ABD-AJ-144_0.csv
1440
                  Time  Activity
0  2018-07-09 00:00:56       0.0
1  2018-07-09 00:01:56       0.0
2  2018-07-09 00:02:56       0.0
3  2018-07-09 00:03:56       0.0
4  2018-07-09 00:04:56       0.0

file No: 


file No: 71
file Name: P_ABD-AJ-0003_20.csv
1440
                  Time  Activity
0  2016-12-02 00:00:30       0.0
1  2016-12-02 00:01:30     260.0
2  2016-12-02 00:02:30    3967.0
3  2016-12-02 00:03:30   10363.0
4  2016-12-02 00:04:30    9905.0

file No: 72
file Name: P_ABD-AJ-0088_13.csv
1440
                  Time  Activity
0  2017-09-28 00:00:06       0.0
1  2017-09-28 00:01:06     247.0
2  2017-09-28 00:02:06       0.0
3  2017-09-28 00:03:06       0.0
4  2017-09-28 00:04:06    1013.0

file No: 73
file Name: P_ABD-AJ-0058_24.csv
1440
                  Time  Activity
0  2017-09-05 00:00:11       0.0
1  2017-09-05 00:01:11       0.0
2  2017-09-05 00:02:11       0.0
3  2017-09-05 00:03:11       0.0
4  2017-09-05 00:04:11     755.0

file No: 74
file Name: P_ABD-AJ-0081_37.csv
1440
                  Time  Activity
0  2017-12-08 00:00:44       6.0
1  2017-12-08 00:01:44       4.0
2  2017-12-08 00:02:44       2.0
3  2017-12-08 00:03:44       7.0
4  2017-12-08 00:04:44       2.0

file No

...


file No: 2634
file Name: P_ABD-AJ-0098_26.csv
1440
                  Time  Activity
0  2017-12-18 00:00:20       0.0
1  2017-12-18 00:01:20       0.0
2  2017-12-18 00:02:20       0.0
3  2017-12-18 00:03:20       0.0
4  2017-12-18 00:04:20       0.0

file No: 2635
file Name: P_ABD-AJ-0013_15.csv
1440
                  Time  Activity
0  2016-12-30 00:00:58       0.0
1  2016-12-30 00:01:58       0.0
2  2016-12-30 00:02:58       0.0
3  2016-12-30 00:03:58       0.0
4  2016-12-30 00:04:58       0.0

file No: 2636
file Name: P_ABD-AJ-0048_11.csv
1440
                  Time  Activity
0  2017-07-13 00:00:00       0.0
1  2017-07-13 00:01:00       0.0
2  2017-07-13 00:02:00    5307.0
3  2017-07-13 00:03:00       0.0
4  2017-07-13 00:04:00       0.0

file No: 2637
file Name: P_ABD-AJ-0041_21.csv
1440
                  Time  Activity
0  2017-06-15 00:00:21       0.0
1  2017-06-15 00:01:21       0.0
2  2017-06-15 00:02:21       0.0
3  2017-06-15 00:03:21       0.0
4  2017-06-15 00:04:21       0.0



file No: 2667
file Name: P_ABD-AJ-0105_2.csv
1440
                  Time  Activity
0  2018-04-05 00:00:09    4970.0
1  2018-04-05 00:01:09     362.0
2  2018-04-05 00:02:09       0.0
3  2018-04-05 00:03:09      10.0
4  2018-04-05 00:04:09      12.0

file No: 2668
file Name: P_ABD-AJ-0116_8.csv
1440
                  Time  Activity
0  2018-04-18 00:00:02       0.0
1  2018-04-18 00:01:02       0.0
2  2018-04-18 00:02:02       0.0
3  2018-04-18 00:03:02       0.0
4  2018-04-18 00:04:02       0.0

file No: 2669
file Name: P_ABD-AJ-0022_21.csv
1440
                  Time  Activity
0  2017-04-19 00:00:02       2.0
1  2017-04-19 00:01:02       0.0
2  2017-04-19 00:02:02     136.0
3  2017-04-19 00:03:02       0.0
4  2017-04-19 00:04:02     192.0

file No: 2670
file Name: P_ABD-AJ-153_2.csv
1440
                  Time  Activity
0  2018-10-26 00:00:39       0.0
1  2018-10-26 00:01:39      44.0
2  2018-10-26 00:02:39       0.0
3  2018-10-26 00:03:39       0.0
4  2018-10-26 00:04:39       0.0

fil


file No: 2700
file Name: P_ABD-AJ-0051_31.csv
1440
                  Time  Activity
0  2017-08-10 00:00:01       0.0
1  2017-08-10 00:01:01       0.0
2  2017-08-10 00:02:01       0.0
3  2017-08-10 00:03:01    8651.0
4  2017-08-10 00:04:01    1878.0

file No: 2701
file Name: P_ABD-AJ-0088_22.csv
1440
                  Time  Activity
0  2017-10-10 00:00:06       0.0
1  2017-10-10 00:01:06       0.0
2  2017-10-10 00:02:06       0.0
3  2017-10-10 00:03:06       0.0
4  2017-10-10 00:04:06       0.0

file No: 2702
file Name: P_ABD-AJ-0003_11.csv
1440
                  Time  Activity
0  2016-12-15 00:00:30     512.0
1  2016-12-15 00:01:30      60.0
2  2016-12-15 00:02:30    2533.0
3  2016-12-15 00:03:30    2009.0
4  2016-12-15 00:04:30     207.0

file No: 2703
file Name: P_ABD-AJ-0045_44.csv
1440
                  Time  Activity
0  2017-08-02 00:00:03       0.0
1  2017-08-02 00:01:03       0.0
2  2017-08-02 00:02:03       0.0
3  2017-08-02 00:03:03       0.0
4  2017-08-02 00:04:03       0.0



file No: 2733
file Name: P_ABD-AJ-0066_5.csv
1440
                  Time  Activity
0  2017-09-20 00:00:00    6782.0
1  2017-09-20 00:01:00       0.0
2  2017-09-20 00:02:00     243.0
3  2017-09-20 00:03:00       2.0
4  2017-09-20 00:04:00       0.0

file No: 2734
file Name: P_ABD-AJ-132_2.csv
1440
              Time  Activity
0  2018-07-10 0:00         0
1  2018-07-10 0:01       487
2  2018-07-10 0:02         0
3  2018-07-10 0:03         0
4  2018-07-10 0:04         0

file No: 2735
file Name: P_ABD-AJ-0069_21.csv
1440
                  Time  Activity
0  2017-09-01 00:00:49       0.0
1  2017-09-01 00:01:49       0.0
2  2017-09-01 00:02:49       0.0
3  2017-09-01 00:03:49       0.0
4  2017-09-01 00:04:49       0.0

file No: 2736
file Name: P_ABD-AJ-0113_14.csv
1440
                  Time  Activity
0  2018-04-07 00:00:16    2049.0
1  2018-04-07 00:01:16       0.0
2  2018-04-07 00:02:16       0.0
3  2018-04-07 00:03:16    3687.0
4  2018-04-07 00:04:16       0.0

file No: 2737
file Name: P


file No: 2767
file Name: P_ABD-AJ-0103_5.csv
1440
                  Time  Activity
0  2018-03-22 00:00:38       0.0
1  2018-03-22 00:01:38       0.0
2  2018-03-22 00:02:38       0.0
3  2018-03-22 00:03:38       0.0
4  2018-03-22 00:04:38       0.0

file No: 2768
file Name: P_ABD-AJ-0012_8.csv
1440
                  Time  Activity
0  2016-12-18 00:00:05       0.0
1  2016-12-18 00:01:05       0.0
2  2016-12-18 00:02:05       0.0
3  2016-12-18 00:03:05       0.0
4  2016-12-18 00:04:05       0.0

file No: 2769
file Name: P_ABD-AJ-0027_1.csv
1440
                  Time  Activity
0  2017-03-24 00:00:01      57.0
1  2017-03-24 00:01:01       4.0
2  2017-03-24 00:02:01      25.0
3  2017-03-24 00:03:01       4.0
4  2017-03-24 00:04:01    6781.0

file No: 2770
file Name: P_ABD-AJ-0020_25.csv
1440
                  Time  Activity
0  2017-03-13 00:00:55      10.0
1  2017-03-13 00:01:55       5.0
2  2017-03-13 00:02:55       0.0
3  2017-03-13 00:03:55       2.0
4  2017-03-13 00:04:55      16.0

fi